In [1]:
import numpy as np

In [25]:
x = np.random.normal(0,1,1000000)
w = np.random.normal(0,1,1000000)

print("Mean of inputs: ", np.mean(x))
print("Standard deviation of inputs: ", np.std(x))
print("\n")
print("Mean of weights: ", np.mean(w))
print("Standard deviation of weights: ", np.std(w))
print("\n")
print("Mean of inputs: ", np.mean(784*x*w))
print("Standard deviation of inputs: ", np.std(784*x*w))

Mean of inputs:  -0.002113822030739876
Standard deviation of inputs:  1.0009610853718647


Mean of weights:  -0.0002697203343350954
Standard deviation of weights:  1.0010529846224274


Mean of inputs:  0.009486774567372234
Standard deviation of inputs:  785.6157360311767


#### Yukarıdaki örnekte görüldüğü üzere inputlar ve weightler için seçim yapılacak dağılımlar için örneklemler oluşturulmuştur. Weigthler birbirlerini etkilemeyecek şekilde seçildiğinden aslında tüm weightlerin 1 dağılım oluşturduğu düşünülebilir. Bu noktada seçilen weight miktarı kadar elde normal dağılım bulunduğu söylenebilir. Aynı şekilde inputların da normal dağılımlardan geldiği düşünüldüğünde aynı şekilde birden çok normal dağılımın elde olduğu söylenebilir. İnput değerimizin 784 tane olduğunu düşünelim. Bu noktada aynı şekilde 784 tane weightimiz bulunmaktadır. Bu noktadan sonra ikiye ayıracak olursak:

<img src="var(xy).png">

- Başlangıçta her node için x*w değerlerinin varyanslarının hesaplanmalarını açıklayalım. Yukarıdaki formule bakıldığı takdirde X ve Y'nin aynı dağılımlara sahip olması ve birbirlerinden bağımsız olmalarından dolayı kovaryansları 0 olarak alınır. Her iki dağılım için de ortalamalar 0 elde edildiği için expected valueları 0 olacaktır. Bu noktada mantık olarak Var(x*y) direkt olarak iki dağılımın varyanslarının çarpımı olarak düşünülebilir.

- Diğer bir nokta her x*w çarpımının toplanıp bir node eklenmesidir. Burada Var(x+y)'nin temelini alacak olursak iki varyansın toplamı eğer covaryansları 0 ise Var(X) + Var(Y) olarak düşünülebilir. Bu noktada covaryansın derinliğine inecek olursak. (x-E(x))*(y-E(y)) ye bakılması gerekmektedir. Dağılımlar aynı olduğu için ve ortalamaları 0 olduğu için direkt olarak x*y olarak düşünülebilir. Bu noktada ise - ve + kısımlar örtüşeceği için kovaryans 0 elde edilir. Bu nedenle tüm nodeların varyanslarının toplanması yeterli olacaktır.

# Ödev 2

In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

from PIL import Image
from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt

import os
import math

In [2]:
PATH = Path("C:/Users/dogan/Desktop/kodluyoruz_bootcamp_102/Hafta5/data/mnist")

In [3]:
def log_softmax(x): 
    return (x.exp()/(x.exp().sum(-1,keepdim=True)) + 1e-20).log()

In [4]:
def nll(preds, actuals): 
    return -preds[range(actuals.shape[0]), actuals].mean()

In [5]:
def validation_acc(model):
    return torch.stack([accuracy(model(xb), yb) for xb, yb in valid_dl]).mean().item()

In [6]:
def loss_func(preds, targets):
    preds = log_softmax(preds)
    return nll(preds, targets)

In [7]:
def accuracy(preds, yb): 
    return (torch.argmax(preds, dim=1, keepdim = True)==yb).float().mean()

In [8]:
def loss_func(preds, targets):
    preds = log_softmax(preds)
    return nll(preds, targets)

In [9]:
def train(model, epochs=5, valid_epoch=5):
    for epoch in range(epochs):
        for xb, yb in train_dl:
            
            preds = model(xb)
            loss = loss_func(preds, yb.squeeze())
            loss.backward()
            optim.step()
            optim.zero_grad()
        
        if epoch % valid_epoch == 0:
            print(validation_acc(model))

In [10]:
def _get_files(p, fs, extensions = None):
    p = Path(p) # to support / notation
    res = [p/f for f in fs if not f.startswith(".") 
           and ((not extensions) or f'.{f.split(".")[-1].lower()}' in extensions)]
    return res

In [11]:
class Dataset():
    def __init__(self, x, y): 
        self.x,self.y = x,y
    def __len__(self): 
        return len(self.x)
    def __getitem__(self, i): 
        return self.x[i].view(-1,1,28,28).cuda(),self.y[i].cuda()


class DataLoader():
    def __init__(self, ds, bs): 
        self.ds, self.bs = ds, bs
    def __iter__(self):
        n = len(self.ds)
        l = torch.randperm(n)

        
        for i in range(0, n, self.bs): 
            idxs_l = l[i:i+self.bs]
            yield self.ds[idxs_l]

In [12]:
def create_ds_from_file(src):
    imgs, labels = [], []
    
    for label in range(10):
        path = src/str(label)
        print(path)
        t = [o.name for o in os.scandir(path)]
        t = _get_files(path, t, extensions = [".jpg", ".png"])
        for e in t:
            l = [np.array(Image.open(e)).reshape(28*28)]
            imgs += l
        labels += ([label] * len(t))
    return torch.tensor(imgs,  dtype=torch.float32), torch.tensor(labels, dtype=torch.long).view(-1,1)

In [13]:
trn_x, trn_y = create_ds_from_file(PATH/"train")

C:\Users\dogan\Desktop\kodluyoruz_bootcamp_102\Hafta5\data\mnist\train\0
C:\Users\dogan\Desktop\kodluyoruz_bootcamp_102\Hafta5\data\mnist\train\1
C:\Users\dogan\Desktop\kodluyoruz_bootcamp_102\Hafta5\data\mnist\train\2
C:\Users\dogan\Desktop\kodluyoruz_bootcamp_102\Hafta5\data\mnist\train\3
C:\Users\dogan\Desktop\kodluyoruz_bootcamp_102\Hafta5\data\mnist\train\4
C:\Users\dogan\Desktop\kodluyoruz_bootcamp_102\Hafta5\data\mnist\train\5
C:\Users\dogan\Desktop\kodluyoruz_bootcamp_102\Hafta5\data\mnist\train\6
C:\Users\dogan\Desktop\kodluyoruz_bootcamp_102\Hafta5\data\mnist\train\7
C:\Users\dogan\Desktop\kodluyoruz_bootcamp_102\Hafta5\data\mnist\train\8
C:\Users\dogan\Desktop\kodluyoruz_bootcamp_102\Hafta5\data\mnist\train\9


In [14]:
val_x,val_y = create_ds_from_file(PATH/"validation")

C:\Users\dogan\Desktop\kodluyoruz_bootcamp_102\Hafta5\data\mnist\validation\0
C:\Users\dogan\Desktop\kodluyoruz_bootcamp_102\Hafta5\data\mnist\validation\1
C:\Users\dogan\Desktop\kodluyoruz_bootcamp_102\Hafta5\data\mnist\validation\2
C:\Users\dogan\Desktop\kodluyoruz_bootcamp_102\Hafta5\data\mnist\validation\3
C:\Users\dogan\Desktop\kodluyoruz_bootcamp_102\Hafta5\data\mnist\validation\4
C:\Users\dogan\Desktop\kodluyoruz_bootcamp_102\Hafta5\data\mnist\validation\5
C:\Users\dogan\Desktop\kodluyoruz_bootcamp_102\Hafta5\data\mnist\validation\6
C:\Users\dogan\Desktop\kodluyoruz_bootcamp_102\Hafta5\data\mnist\validation\7
C:\Users\dogan\Desktop\kodluyoruz_bootcamp_102\Hafta5\data\mnist\validation\8
C:\Users\dogan\Desktop\kodluyoruz_bootcamp_102\Hafta5\data\mnist\validation\9


In [15]:
train_ds = Dataset(trn_x, trn_y)
valid_ds = Dataset(val_x,val_y)

In [16]:
train_dl = DataLoader(train_ds, 256)
valid_dl = DataLoader(valid_ds, 256)

In [17]:
def stats(x):
    print(f"Mean: {x.mean()}, Std: {x.std()}")

In [18]:
class Func(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x): 
        return self.func(x)

In [19]:
def flatten(x):      
    return x.view(x.shape[0], -1)

In [20]:
def print_t(x):      
    print(x.shape)
    return x

In [21]:
def gain(a):
    return math.sqrt(2.0 / (1 + a**2))

In [22]:
# kaiming for uniform init
def kaiming_uniform(x, a):
    n = x[0].shape.numel()
    std = gain(a) / math.sqrt(n)
    bound = math.sqrt(3.) * std
    x.data.uniform_(-bound, bound)

In [27]:
l1 = nn.Conv2d(1, 8, 5, padding=2,stride=2)
kaiming_uniform(l1.weight, a = 0.4)

In [28]:
model = nn.Sequential(
        l1, nn.ReLU(), #14
        nn.Conv2d(8, 16, 3, padding=1,stride=2), nn.ReLU(), # 7
        nn.Conv2d(16, 27, 3, padding=1,stride=2), nn.ReLU(), # 4
#         nn.Conv2d(20, 32, 3, padding=1,stride=2), nn.ReLU(), # 2
        #Func(print_t),
        nn.AdaptiveMaxPool2d(1),
        Func(flatten),
        nn.Linear(27,10)
).cuda()

In [29]:
optim = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-3)

In [30]:
train(model,80,10)

0.5555959343910217
0.9382267594337463
0.9535791873931885
0.9659338593482971
0.9603015780448914
0.9652979373931885
0.9683866500854492
0.9688408374786377
